    # Experiment 3
All units are in mm

In [7]:
import numpy as np
import pandas as pd

In [8]:
a = 4.04e-10

In [9]:
D = 20 # mm
dD = 1 # mm

In [10]:
# Converts arbitrary units into mm
ratio = 49.3 / 6.78 

In [11]:
dy = 2 / 2 * ratio
dz = 2.5 / 2 * ratio

In [12]:
print(
    dy * ratio,
    dz * ratio
)

52.873060624254904 66.09132578031863


In [13]:
# Not all measurements have an additional uncertainty of the centre point
# Measurement y dy z dz 
measurements = [
    ('A', 0.1, 0.05, 2.35, 0.1,),
    ('B', 0.6, 0.2, 2.25, 0.1,),
    ('C', 1.4, 0.2,  2.9, 0.1,),
    ('D', 1.1, 0.2, 1.3, 0.2, ),
    ('E', 1.8, 0.15, 2.05, 0.1),
]

In [14]:
columns = ['Point', 'y', 'dy', 'z', 'dz']
laue = pd.DataFrame(measurements, columns=columns) 

In [15]:
laue = laue.applymap(lambda x: x * ratio if not isinstance(x, str) else x)

/tmp/ipykernel_17680/662506833.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  laue = laue.applymap(lambda x: x * ratio if not isinstance(x, str) else x)


In [16]:
laue['yz'] = laue.y / laue.z

In [17]:
laue['dyz'] = laue.yz * ((laue.dy + dy) / laue.y + (laue.dz + dz) / laue.z)

In [18]:
laue['theta_exp'] = 1/2 * np.degrees(np.arctan(np.sqrt(laue.y ** 2 + laue.z ** 2) / D))

In [19]:
laue['dtheta_exp'] = 1/4 * np.degrees(
    abs(
        (np.sqrt((laue.y + laue.dy + dy) ** 2 + (laue.z + laue.dz + dz) ** 2) / (D - dD)) - (np.sqrt((laue.y - laue.dy - dy) ** 2 + (laue.z - laue.dz - dz) ** 2) / (D + dD))
    )
)

In [20]:
laue.yz

0    0.042553
1    0.266667
2    0.482759
3    0.846154
4    0.878049
Name: yz, dtype: float64

In [21]:
laue['kl'] = [
    0, 
    1/3,
    1/2, 
    1,
    1,
]

In [22]:
laue['plane'] = [
    '204',
    '113',
    '224',
    '133',
    '333'
]

In [23]:
expected = []
for h, k, l in laue.plane:
    h = int(h)
    k = int(k)
    l = int(l)
    expected.append(
        np.degrees(np.arcsin(h / np.sqrt(h ** 2 + k ** 2 + l ** 2)))
    )
laue['theta_thy'] = expected

In [24]:
laue[['theta_exp', 'dtheta_exp', 'theta_thy']]

,theta_exp,dtheta_exp,theta_thy
0,20.267887,14.398863,26.565051
1,20.125910,16.699176,17.548401
2,24.749202,19.560359,24.094843
3,15.881575,18.758469,13.262676
4,22.382804,19.938084,35.264390


In [25]:
wavelength = [
    2 * a * np.sin(np.radians(t)) / np.sqrt(int(h) ** 2 + int(k) ** 2 + int(l) ** 2) for (h, k, l), t in zip(laue.plane, laue.theta_exp)
]
dwavelength = [
    np.radians(dt) * 2 * a * np.cos(np.radians(t)) / np.sqrt(int(h) ** 2 + int(k) ** 2 + int(l) ** 2) for (h, k, l), t, dt in zip(laue.plane, laue.theta_exp, laue.dtheta_exp)
]

In [26]:
laue['wavelength'] = wavelength 
laue['dwavelength'] = dwavelength 

In [27]:
laue

,Point,y,dy,z,dz,yz,dyz,theta_exp,dtheta_exp,kl,plane,theta_thy,wavelength,dwavelength
0,A,0.727139,0.363569,17.087758,0.727139,0.042553,0.471254,20.267887,14.398863,0.000000,204,26.565051,6.258737e-11,4.259350e-11
1,B,4.362832,1.454277,16.360619,0.727139,0.266667,0.693333,20.125910,16.699176,0.333333,113,17.548401,8.382623e-11,6.666912e-11
2,C,10.179941,1.454277,21.087021,0.727139,0.482759,0.638526,24.749202,19.560359,0.500000,224,24.094843,6.904843e-11,5.113486e-11
3,D,7.998525,1.454277,9.452802,1.454277,0.846154,1.866864,15.881575,18.758469,1.000000,133,13.262676,5.072592e-11,5.837237e-11
4,E,13.088496,1.090708,14.906342,0.727139,0.878049,1.139203,22.382804,19.938084,1.000000,333,35.264390,5.921317e-11,5.003484e-11


In [28]:
latex = laue.copy()
latex.wavelength = latex.wavelength / 1e-12
latex.dwavelength = latex.dwavelength / 1e-12
latex.columns = [
    'Point',
    r'$y$',
    r'$\Delta y$',
    r'$z$',
    r'$\Delta z$',
    r'$\frac{y}{z}$',
    r'$\Delta\frac{y}{z}$',
    r'$\theta_{exp} (\deg)$',
    r'$\Delta \theta_{exp} (\deg)$',
    r'$\frac{k}{l}$',
    'Plane',
    r'$\theta_{thy} (\deg)$',
    r'$\lambda (pm)$',
    r'$\Delta \lambda (pm)$'
]
latex = latex.to_latex(index=False, float_format='{:.2f}'.format, column_format='|'.join('c' * 14))
for old, new in [
    (r'\toprule', r'\hline'),
    (r'\midrule', r'\hline'),
    (r'\bottomrule', r'\hline')
]:
    latex = latex.replace(old, new)
    
print(latex)

\begin{tabular}{c|c|c|c|c|c|c|c|c|c|c|c|c|c}
\hline
Point & $y$ & $\Delta y$ & $z$ & $\Delta z$ & $\frac{y}{z}$ & $\Delta\frac{y}{z}$ & $\theta_{exp} (\deg)$ & $\Delta \theta_{exp} (\deg)$ & $\frac{k}{l}$ & Plane & $\theta_{thy} (\deg)$ & $\lambda (pm)$ & $\Delta \lambda (pm)$ \\
\hline
A & 0.73 & 0.36 & 17.09 & 0.73 & 0.04 & 0.47 & 20.27 & 14.40 & 0.00 & 204 & 26.57 & 62.59 & 42.59 \\
B & 4.36 & 1.45 & 16.36 & 0.73 & 0.27 & 0.69 & 20.13 & 16.70 & 0.33 & 113 & 17.55 & 83.83 & 66.67 \\
C & 10.18 & 1.45 & 21.09 & 0.73 & 0.48 & 0.64 & 24.75 & 19.56 & 0.50 & 224 & 24.09 & 69.05 & 51.13 \\
D & 8.00 & 1.45 & 9.45 & 1.45 & 0.85 & 1.87 & 15.88 & 18.76 & 1.00 & 133 & 13.26 & 50.73 & 58.37 \\
E & 13.09 & 1.09 & 14.91 & 0.73 & 0.88 & 1.14 & 22.38 & 19.94 & 1.00 & 333 & 35.26 & 59.21 & 50.03 \\
\hline
\end{tabular}


In [29]:
E = 35 * 1e3 * 1.602e-10
l = 6.626e-34 * 3e8 / E / 1e-12
print(l)

3.545211342964152e-08
